# 2023-11-20

In [1]:
from haystack.pipelines import Pipeline

/home/silvhua/solarathon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:


p = Pipeline()
p.add_node(component=text_converter, name="TextConverter", inputs=["File"])
p.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"]) # https://docs.haystack.deepset.ai/docs/preprocessor
p.add_node(component=document_classifier, name="DocumentClassifier", inputs=["PreProcessor"])
p.add_node(component=document_store, name="DocumentStore", inputs=["DocumentClassifier"])
p.add_node(component=retriever, name="Retriever", inputs=["PreProcessor"])

# p.run(file_paths=["filename.txt"])

NameError: name 'text_converter' is not defined

In [3]:
from haystack.nodes import TextConverter
p = Pipeline()
text_converter = TextConverter()
p.add_node(component=text_converter, name="TextConverter", inputs=["File"])

In [4]:
p

# 2023-11-21

## Iteration 1

In [1]:
from haystack.pipelines import Pipeline
from haystack.nodes import TextConverter
from haystack.nodes import PreProcessor
from haystack.document_stores import FAISSDocumentStore

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [8]:

text_converter = TextConverter() # https://docs.haystack.deepset.ai/docs/file_converters

preprocessor = PreProcessor( # https://docs.haystack.deepset.ai/docs/preprocessor
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
  	remove_substrings=None,
    split_by=None, # Unit for splitting the document. Can be "word", "sentence", or "passage". Set to None to disable splitting.
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=0,
  	max_chars_check = None
)

In [3]:
from datetime import datetime

def append_timestamp(string):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
    return f'{string}_{timestamp}' 

index_filename = append_timestamp('journal_article_index')
config_filename = append_timestamp('journal_article_config')
path = '../data'
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
document_store.save(index_path=f'{path}/{index_filename}', config_path=f'{path}/{config_filename}')

In [7]:


p = Pipeline()
p.add_node(component=text_converter, name="TextConverter", inputs=["File"])
p.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
p.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

p.run(file_paths=[f"{path}/journal_article.txt"])

Converting files: 100%|██████████| 1/1 [00:00<00:00, 110.42it/s]


Exception: Exception while running node 'PreProcessor': '>' not supported between instances of 'int' and 'NoneType'
Enable debug logging to see the data that was passed when the pipeline failed.

## Iteration 2

In [1]:
from haystack.pipelines import Pipeline
from haystack.nodes import TextConverter
from haystack.nodes import PreProcessor
from haystack.document_stores import FAISSDocumentStore

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


In [10]:

text_converter = TextConverter() # https://docs.haystack.deepset.ai/docs/file_converters

preprocessor = PreProcessor( # https://docs.haystack.deepset.ai/docs/preprocessor
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
  	remove_substrings=None,
    split_by='word', # Unit for splitting the document. Can be "word", "sentence", or "passage". Set to None to disable splitting.
    split_length=200,
    split_respect_sentence_boundary=True,
    split_overlap=0,
  	max_chars_check = 10000
)

In [11]:
from datetime import datetime

def append_timestamp(string):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
    return f'{string}_{timestamp}' 

index_filename = append_timestamp('journal_article_index')
config_filename = append_timestamp('journal_article_config')
path = '../data'
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")
document_store.save(index_path=f'{path}/{index_filename}', config_path=f'{path}/{config_filename}')

In [12]:


p = Pipeline()
p.add_node(component=text_converter, name="TextConverter", inputs=["File"])
p.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
p.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

p.run(file_paths=[f"{path}/journal_article.txt"])


Converting files: 100%|██████████| 1/1 [00:00<00:00, 57.32it/s]

Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 75.31docs/s]
Writing Documents: 10000it [00:00, 77521.42it/s]         


{'documents': [<Document: {'content': 'High doses of anti-inflammatory drugs compromise muscle strength and hypertrophic adaptations to resistance training in young adults.\n\nDiscussion\nThe interest in this study was spurred by the intriguing observation that while NSAIDs might have a negative impact on acute exercise responses (satellite cell activity, translational signalling and protein synthesis), previous human studies have failed to demonstrate a detrimental effect of NSAIDs on the development of muscle hypertrophy in response to chronic resistance training in young adults, possibly due to differences in drug dosage across studies. Accordingly, in the current study, healthy young men and women performed 8 weeks of supervised resistance training with concomitant high- or low-dose NSAID treatment. The major and novel findings were that 1) maximal overthe-counter doses of ibuprofen compromised resistance exercise-induced muscle hypertropy\nindependent of training mode; 2) increase

In [13]:
!mv faiss_document_store.db ../data/testing

# End of Page